In [1]:
# 2020 Q1
import pandas as pd
import numpy as np
import datetime
import os
import glob
import gc
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2020_Q1/Post_YoY'

In [2]:
store_list_files=glob.glob("/home/jian/BigLots/static_files/Store_list/MediaStormStores20*.txt")
store_list_files=sorted(store_list_files,reverse=True)
latest_store_list=store_list_files[0]
store_list_files.remove(latest_store_list)

print(latest_store_list)
store_list_files

/home/jian/BigLots/static_files/Store_list/MediaStormStores20200501-152115-660.txt


['/home/jian/BigLots/static_files/Store_list/MediaStormStores20200401-135137-445.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20200301-134228-900.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20200201-134613-934.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20200101-135601-098.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20191201-133313-746.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20191101-134011-956.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20191001-134050-499.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190901-135942-508.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190801-135940-752.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190701-134908-815.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190601-134302-813.txt',
 '/home/jian/BigLots/static_files/Store_lis

In [3]:
store_info=pd.read_table(latest_store_list,sep="|",dtype=str)
store_info=store_info[['location_id','city_nm','state_nm','zip_cd','open_dt']]

for file in store_list_files:
    df=pd.read_table(file,sep="|",dtype=str,usecols=['location_id','city_nm','state_nm','zip_cd','open_dt'])
    df=df[~df['location_id'].isin(store_info['location_id'].tolist())]
    store_info=store_info.append(df)
    

store_info['zip_cd']=store_info['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))


DMA_Zip=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,dtype=str)
DMA_Zip=DMA_Zip.iloc[:,[0,2]]
DMA_Zip.columns=['zip_cd','DMA']
DMA_Zip=DMA_Zip.drop_duplicates(['zip_cd'])


store_info=pd.merge(store_info,DMA_Zip,on="zip_cd",how="left")

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
weeks_2020Q1=[datetime.date(2020,2,8)+datetime.timedelta(days=x*7) for x in range(13)]
weeks_2019Q1=[x-datetime.timedelta(days=52*7) for x in weeks_2020Q1]
weeks_2019Q1_str_date=[str(x) for x in weeks_2019Q1]
weeks_2019Q1_str_digital=[str(x).replace("-","") for x in weeks_2019Q1]

In [5]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [6]:
Q1_files_2019=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
Q1_files_2019=[x for x in Q1_files_2019 if x.split("diaStormDailySalesHistory")[1][:8] in [str(x) for x in weeks_2019Q1_str_digital]]
print(len(Q1_files_2019))

Q1_files_2019.sort()

1


In [7]:
folder_2019_by_week="/home/jian/BigLots/"

daily_sale_files=list(recursive_file_gen(folder_2019_by_week))
daily_sale_files=[x for x in daily_sale_files if "aily" in x]
# Q1_files_2019=[]
for week_end_date in weeks_2019Q1_str_date[1:]:
    file_path=[x for x in daily_sale_files if str(week_end_date) in x]
    Q1_files_2019=Q1_files_2019+file_path

In [8]:
print(len(Q1_files_2019))
Q1_files_2019

13


['/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20190209.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-16/MediaStormDailySales20190219-113605-481.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-23/MediaStormDailySales20190226-112611-940.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-02/MediaStormDailySales20190305-112738-899.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-09/MediaStormDailySales20190312-121205-996.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-16/MediaStormDailySales20190319-112334-057.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-23/MediaStormDailySales20190326-112740-903.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-30/MediaStormDailySales20190402-112556-551.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-04-06/MediaStormDailySales20190409-122759-093.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-04-13/MediaStormDailySa

In [9]:
Q1_files_2020=[]
for week_end_date in weeks_2020Q1:
    file_path=[x for x in daily_sale_files if str(week_end_date) in x]
    Q1_files_2020=Q1_files_2020+file_path
    
print(len(Q1_files_2020))
Q1_files_2020=sorted(Q1_files_2020)
Q1_files_2020

13


['/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-02-08/MediaStormDailySales20200211-120911-483.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-02-15/MediaStormDailySales20200218-110927-085.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-02-22/MediaStormDailySales20200225-111349-615.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-02-29/MediaStormDailySales20200303-111359-260.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-03-07/MediaStormDailySales20200311-125005-860.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-03-14/MediaStormDailySales20200317-112349-402.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-03-21/MediaStormDailySales20200324-112302-496.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-03-28/MediaStormDailySales20200331-112303-873.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-04-04/MediaStormDailySales20200407-110641-771.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-04-11/MediaStormDailyS

In [10]:
def week_end_dt(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=(5-x.weekday()))
    return y


In [11]:
def load_and_agg_df(file_list):
    
    i_counter=0

    sales_agg_df=pd.DataFrame()
    ids_by_week_store=pd.DataFrame()

    for file_path in file_list:
        df=pd.read_table(file_path,dtype=str,sep="|",usecols=None,nrows=None)
        df=df[df['location_id']!="6990"]
        # print(df.shape)
        df=df.drop_duplicates()

        df['transaction_dt']=df['transaction_dt'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").date())
        df['item_transaction_amt']=df['item_transaction_amt'].astype(float)

        df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")

        date_max=df['transaction_dt'].max()
        date_min=df['transaction_dt'].min()

        

        if ((date_max-date_min).days==6) & (date_max.weekday()==5):
            print(i_counter,date_max,date_min,datetime.datetime.now())
        else:
            print("Dates in the data not 7 days but included ",file_path)
            print("Please check the reason why no sales data in the week for the physical stores")
            
            
        df['week_end_dt']=date_max
        df_agg_sales=df.groupby(['location_id','week_end_dt','rewards_label'])['item_transaction_amt'].sum().to_frame().reset_index()
        df_agg_sales=df_agg_sales.rename(columns={"item_transaction_amt":"sales"})

        df=df[df['item_transaction_amt']>0]
        df_agg_trans=df[['location_id','transaction_dt','week_end_dt','transaction_id','customer_id_hashed','rewards_label']].drop_duplicates()
        df_agg_trans['transactions']=1
        df_agg_trans=df_agg_trans.groupby(['location_id','week_end_dt','rewards_label'])['transactions'].sum().to_frame().reset_index()

        df_agg_sales=pd.merge(df_agg_sales,df_agg_trans,on=['location_id','week_end_dt','rewards_label'],how="outer")

        df=df[df['rewards_label']=="Rewards"]
        df=df[['location_id','week_end_dt','customer_id_hashed']].drop_duplicates()

        
                

        sales_agg_df=sales_agg_df.append(df_agg_sales)
        ids_by_week_store=ids_by_week_store.append(df)
        i_counter+=1


    return sales_agg_df,ids_by_week_store

In [12]:
sales_agg_2020, ids_by_week_store_2020 = load_and_agg_df(Q1_files_2020)

0 2020-02-08 2020-02-02 2020-05-13 21:07:05.543418
1 2020-02-15 2020-02-09 2020-05-13 21:11:09.265944
2 2020-02-22 2020-02-16 2020-05-13 21:15:03.565423
3 2020-02-29 2020-02-23 2020-05-13 21:19:22.548600
4 2020-03-07 2020-03-01 2020-05-13 21:24:20.951000
5 2020-03-14 2020-03-08 2020-05-13 21:31:31.752135
6 2020-03-21 2020-03-15 2020-05-13 21:38:14.196278
7 2020-03-28 2020-03-22 2020-05-13 21:42:27.051102
8 2020-04-04 2020-03-29 2020-05-13 21:46:49.368948
9 2020-04-11 2020-04-05 2020-05-13 21:51:40.084881
Dates in the data not 7 days but included  /home/jian/BigLots/2020_by_weeks/MediaStorm_2020-04-18/MediaStormDailySales20200421-111910-356.txt
Please check the reason why no sales data in the week for the physical stores
11 2020-04-25 2020-04-19 2020-05-13 22:01:41.679614
12 2020-05-02 2020-04-26 2020-05-13 22:07:43.661157


In [13]:
sales_agg_2019, ids_by_week_store_2019 = load_and_agg_df(Q1_files_2019)

0 2019-02-09 2019-02-03 2020-05-13 22:12:18.486963
1 2019-02-16 2019-02-10 2020-05-13 22:16:11.814896
2 2019-02-23 2019-02-17 2020-05-13 22:20:17.681284
3 2019-03-02 2019-02-24 2020-05-13 22:24:58.084207
4 2019-03-09 2019-03-03 2020-05-13 22:29:31.444985
5 2019-03-16 2019-03-10 2020-05-13 22:34:07.212818
6 2019-03-23 2019-03-17 2020-05-13 22:38:53.000378
7 2019-03-30 2019-03-24 2020-05-13 22:43:22.020877
8 2019-04-06 2019-03-31 2020-05-13 22:48:46.746244
9 2019-04-13 2019-04-07 2020-05-13 22:54:25.365131
10 2019-04-20 2019-04-14 2020-05-13 22:59:34.235693
11 2019-04-27 2019-04-21 2020-05-13 23:03:48.678842
12 2019-05-04 2019-04-28 2020-05-13 23:08:32.374788


In [14]:
unique_id_Ly_by_store=ids_by_week_store_2019.groupby('location_id')['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"Q4_unique_id"})
unique_id_Ty_by_store=ids_by_week_store_2020.groupby('location_id')['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"Q4_unique_id"})
unique_id_Ly_by_store['rewards_label']="Rewards"
unique_id_Ty_by_store['rewards_label']="Rewards"

unique_id_Ly_by_store_week=ids_by_week_store_2019.groupby(['location_id','week_end_dt'])['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"Q4_unique_id"})
unique_id_Ty_by_store_week=ids_by_week_store_2020.groupby(['location_id','week_end_dt'])['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"Q4_unique_id"})

unique_id_Ly_by_store_week['rewards_label']="Rewards"
unique_id_Ty_by_store_week['rewards_label']="Rewards"

In [15]:
sales_agg_Ly_with_id_by_week=pd.merge(sales_agg_2019,unique_id_Ly_by_store_week,on=['location_id','week_end_dt','rewards_label'],how="outer")
sales_agg_Ty_with_id_by_week=pd.merge(sales_agg_2020,unique_id_Ty_by_store_week,on=['location_id','week_end_dt','rewards_label'],how="outer")


In [16]:
sales_agg_Ly_with_id_by_week['location_id']=sales_agg_Ly_with_id_by_week['location_id'].astype(int)
sales_agg_Ty_with_id_by_week['location_id']=sales_agg_Ty_with_id_by_week['location_id'].astype(int)
unique_id_Ly_by_store['location_id']=unique_id_Ly_by_store['location_id'].astype(int)
unique_id_Ty_by_store['location_id']=unique_id_Ty_by_store['location_id'].astype(int)

In [17]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2020_Q1/Post_YoY/BL_2020_Q1_post_data_positive_sales_trans_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
sales_agg_Ly_with_id_by_week.to_excel(writer,"sales_by_week_LastYear",index=False)
sales_agg_Ty_with_id_by_week.to_excel(writer,"sales_by_week_ThisYear",index=False)
unique_id_Ly_by_store.to_excel(writer,"ids_by_store_quarter_LastYear",index=False)
unique_id_Ty_by_store.to_excel(writer,"ids_by_store_quarter_ThisYear",index=False)
writer.save()

In [18]:
print(datetime.datetime.now())

2020-05-13 23:16:58.954357


In [19]:
ids_by_week_store_2019.to_csv("/home/jian/Projects/Big_Lots/Analysis/2020_Q1/Post_YoY/BL_2019_Q1_ids_2019Q1_by_store_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
ids_by_week_store_2020.to_csv("/home/jian/Projects/Big_Lots/Analysis/2020_Q1/Post_YoY/BL_2020_Q1_ids_2020Q1_by_store_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
